<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%9C%D0%BE%D0%BB%D0%B5%D0%BA%D1%83%D0%BB%D0%B0_%D0%B1%D0%B8%D0%BE_%D1%83%D1%80%D0%BE%D0%BA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

In [ ]:
from Bio import Entrez

In [ ]:
from Bio import SeqIO

In [ ]:
Entrez.email="A.N.Other@example.com"

In [ ]:
import pandas as pd
import io
import xml.etree.ElementTree as ET

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
dis = pd.read_table(io.BytesIO(uploaded['gene_disease_associations.tsv']))
dis.head(10)

In [ ]:
diseaseName=pd.unique(dis['diseaseName'])

In [ ]:
print(diseaseName)

In [ ]:
dis.shape

In [ ]:
dis['diseaseName'].value_counts()

In [ ]:
liver=dis.loc[dis['diseaseName'] == 'Liver Cirrhosis, Experimental']
liver # выбираем заболевание для работы - Liver Cirrhosis, Experimental (Цирроз печени, экспериментальный)

In [ ]:
# анализируем ген A2M, ID = 2
handle=Entrez.efetch(db='gene',id='2',rettype='xml')
gene_locus=ET.parse(handle).getroot().find('Entrezgene/Entrezgene_locus')
region=gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
start=int(region.find("Seq-interval_from").text)+1
end=int(region.find("Seq-interval_to").text)+1
gi_id=region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
strand=region.find("Seq-interval_strand/Na-strand").get("value")
print("start: ",start,"end: ",end,"id: ",gi_id,"нить: ",strand)

In [ ]:
handle=Entrez.esummary(db="gene", id=2, rettype="xml")
record = Entrez.read(handle)
record['DocumentSummarySet']['DocumentSummary'][0]['Name']

In [ ]:
id_chrom = record['DocumentSummarySet']['DocumentSummary'][0]['GenomicInfo'][0]['ChrAccVer']
id_chrom

In [ ]:
handle = Entrez.esummary(db="gene", id=2, rettype = 'xml')
record = Entrez.read(handle)

In [ ]:
chrom_num = record['DocumentSummarySet']['DocumentSummary'][0]['Chromosome']
chrom_num

In [ ]:
handle = Entrez.efetch(db="nucleotide",
                       id="NC_000012.12",
                       rettype = 'fasta',
                       retmode = 'text')
#загружаем
chromosome = SeqIO.read(handle, 'fasta')

In [ ]:
chromosome

In [ ]:
chromosome.description

In [ ]:
geneA2M = chromosome.seq[9067708:9116229]
print(geneA2M)
len(geneA2M)

In [ ]:
liver_cirrhosis_db = dis.loc[dis['diseaseName'] == "Liver Cirrhosis, Experimental"]
liver_cirrhosis_db

In [ ]:
gene_id_lst=liver_cirrhosis_db['geneId'].tolist()
gene_id_lst

In [ ]:
gene_info = liver_cirrhosis_db[['geneId','geneSymbol']]
gene_info

In [ ]:
gene_info_10=gene_info.head(10)

In [ ]:
# функция сборки информации для каждого гена болезни Liver Cirrhosis

def get_gene_info(row):
  gene_id = row['geneId']
  handle = Entrez.efetch(db="gene", id=gene_id, rettype = 'xml')
  gene_locus=ET.parse(handle).getroot().find('Entrezgene/Entrezgene_locus')
  region=gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
  gene_locus.find("Gene-commentary/Gene-commentary_seqs/Seq-loc/Seq-loc_int/Seq-interval")
  start=int(region.find("Seq-interval_from").text)+1
  end=int(region.find("Seq-interval_to").text)+1
  gi_id=region.find("Seq-interval_id/Seq-id/Seq-id_gi").text
  strand=region.find("Seq-interval_strand/Na-strand").get("value")

  handle = Entrez.esummary(db="gene", id=gene_id, rettype = 'xml')
  record = Entrez.read(handle)
  chrom_num = record['DocumentSummarySet']['DocumentSummary'][0]['Chromosome']
  mapLoc = record['DocumentSummarySet']['DocumentSummary'][0]['MapLocation']
  ID_chr = record['DocumentSummarySet']['DocumentSummary'][0]['GenomicInfo'][0]['ChrAccVer']

  handle = Entrez.efetch(db="nucleotide",
                       id=ID_chr,
                       rettype = 'fasta',
                       retmode = 'text')

  chromosome = SeqIO.read(handle, 'fasta')
  gene = chromosome.seq[start: end]

  row['start'], row['end'], row['gi_id'], row['strand'], row['chrom_num'],\
  row['mapLoc'], row['ID chromosome'], row['gene lenght'], row['gene'] = start, end, gi_id, strand, chrom_num, mapLoc, ID_chr, len(gene), gene

  return row


In [ ]:
gene_info_10 = gene_info.apply(get_gene_info, axis=1)
gene_info_10